In [20]:

import os
import random
import sys
from glob import glob
import re
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from tqdm import tqdm
from time import time
import torch
import torch.utils.data as data

import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from efficientnet_pytorch import EfficientNet
import copy
from adamp import AdamP
import matplotlib.pyplot as plt
import seaborn as sns

print ("PyTorch version:[%s]."%(torch.__version__))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print ("device:[%s]."%(device))

def seed_everything(seed):
    """
    동일한 조건으로 학습을 할 때, 동일한 결과를 얻기 위해 seed를 고정시킵니다.
    
    Args:
        seed: seed 정수값
    """
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

seed_everything(42)


IMG_EXTENSIONS = [
    ".jpg", ".JPG", ".jpeg", ".JPEG", ".png",
    ".PNG", ".ppm", ".PPM", ".bmp", ".BMP",
]


def is_image_file(filename):
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)

class MaskLabels:
    mask = 0
    incorrect = 1
    normal = 2

class GenderLabels:
    male = 0
    female = 1


class AgeGroup:
    map_label = lambda x: 0 if int(x) < 30 else 1 if int(x) < 58 else 2


class MaskBaseDataset(data.Dataset):
    num_classes = 3 * 2 * 3

    _file_names = {
        "mask1": MaskLabels.mask,
        "mask2": MaskLabels.mask,
        "mask3": MaskLabels.mask,
        "mask4": MaskLabels.mask,
        "mask5": MaskLabels.mask,
        "incorrect_mask": MaskLabels.incorrect,
        "normal": MaskLabels.normal
    }

    image_paths = []
    mask_labels = []
    gender_labels = []
    age_labels = []

    def __init__(self, img_dir, transform=None):
        """
        MaskBaseDataset을 initialize 합니다.

        Args:
            img_dir: 학습 이미지 폴더의 root directory 입니다.
            transform: Augmentation을 하는 함수입니다.
        """
        self.img_dir = img_dir
        self.transform = transform

        self.setup()

    def set_transform(self, transform):
        """
        transform 함수를 설정하는 함수입니다.
        """
        self.transform = transform
        
    def setup(self):
        """
        image의 경로와 각 이미지들의 label을 계산하여 저장해두는 함수입니다.
        """
        profiles = os.listdir(self.img_dir)
        for profile in profiles:
            if profile.startswith("."):  # "." 로 시작하는 파일은 무시합니다
                continue
            img_folder = os.path.join(self.img_dir, profile)
            for file_name in os.listdir(img_folder):
                _file_name, ext = os.path.splitext(file_name)
                if _file_name not in self._file_names:  # "." 로 시작하는 파일 및 invalid 한 파일들은 무시합니다
                    continue
                img_path = os.path.join(self.img_dir, profile, file_name)  # (resized_data, 000004_male_Asian_54, mask1.jpg)
                self.image_paths.append(img_path)
                mask_label = self._file_names[_file_name]
                self.mask_labels.append(mask_label)
                id, gender, race, age = profile.split("_")
                gender_label = getattr(GenderLabels, gender)
                age_label = AgeGroup.map_label(age)

                self.gender_labels.append(gender_label)
                self.age_labels.append(age_label)
        '''
        추가 데이터를 불러오는 라인이지만, 외부데이터 추가시 f1 score가 더 안나오는 현상이 발생하여 뺌
        profiles = os.listdir('./input/data/train/addition')
        for profile in profiles:
            img_folder = os.path.join('./input/data/train/addition', profile)
            for file_name in os.listdir(img_folder):
                if file_name.startswith("."):
                    continue
                img_path = os.path.join('./input/data/train/addition', profile, file_name)
                self.image_paths.append(img_path)
                id,gender,race,age,mask_label = profile.split("_")
                gender_label = getattr(GenderLabels, gender)
                age_label = AgeGroup.map_label(age)
                self.mask_labels.append(int(mask_label))
                self.gender_labels.append(gender_label)
                self.age_labels.append(age_label)
        '''
    def __getitem__(self, index):
        """
        데이터를 불러오는 함수입니다. 
        데이터셋 class에 데이터 정보가 저장되어 있고, index를 통해 해당 위치에 있는 데이터 정보를 불러옵니다.
        Args:
            index: 불러올 데이터의 인덱스값입니다.
        """
        # 이미지를 불러옵니다.
        image_path = self.image_paths[index]
        image = Image.open(image_path)
        
        # 레이블을 불러옵니다.
        mask_label = self.mask_labels[index]
        gender_label = self.gender_labels[index]
        age_label = self.age_labels[index]
        # 각각의 label들을 모두 return해주어 각각의 loss를 따로 구한다.
        # 이미지를 Augmentation 시킵니다.
        image_transform = self.transform(image)
        return image_transform, mask_label,gender_label,age_label

    def __len__(self):
        return len(self.image_paths)


class cfg:
    data_dir = '../input/data/train'
    img_dir = f'{data_dir}/images'
    df_path = f'{data_dir}/train.csv'


In [7]:
import torch.nn.init as init

def initialize_weights(model):
    """
    Initialize all weights using xavier uniform. 
    For more weight initialization methods, check https://pytorch.org/docs/stable/nn.init.html
    """
    for m in model.modules():
        if isinstance(m, nn.Conv2d):
            init.xavier_uniform_(m.weight.data)
            if m.bias is not None:
                m.bias.data.zero_()
        elif isinstance(m, nn.BatchNorm2d):
            m.weight.data.fill_(1)
            m.bias.data.zero_()
        elif isinstance(m, nn.Linear):
            m.weight.data.normal_(0, 0.01)
            m.bias.data.zero_()

In [8]:
import math
import torch.optim.lr_scheduler as lr_scheduler

class CosineAnnealingWarmUpRestart(lr_scheduler._LRScheduler):
    def __init__(self, optimizer, T_0, T_mult=1, eta_max=0.1, T_up=0, gamma=1., last_epoch=-1):
        if T_0 <= 0 or not isinstance(T_0, int):
            raise ValueError("Expected positive integer T_0, but got {}".format(T_0))
        if T_mult < 1 or not isinstance(T_mult, int):
            raise ValueError("Expected integer T_mult >= 1, but got {}".format(T_mult))
        if T_up < 0 or not isinstance(T_up, int):
            raise ValueError("Expected positive integer T_up, but got {}".format(T_up))
        self.T_0 = T_0
        self.T_mult = T_mult
        self.base_eta_max = eta_max
        self.eta_max = eta_max
        self.T_up = T_up
        self.T_i = T_0
        self.gamma = gamma
        self.cycle = 0
        self.T_cur = last_epoch
        super(CosineAnnealingWarmUpRestart, self).__init__(optimizer, last_epoch)
    
    def get_lr(self):
        if self.T_cur == -1:
            return self.base_lrs
        elif self.T_cur < self.T_up:
            return [(self.eta_max - base_lr)*self.T_cur / self.T_up + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.eta_max - base_lr) * (1 + math.cos(math.pi * (self.T_cur-self.T_up) / (self.T_i - self.T_up))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.T_cur = self.T_cur + 1
            if self.T_cur >= self.T_i:
                self.cycle += 1
                self.T_cur = self.T_cur - self.T_i
                self.T_i = (self.T_i - self.T_up) * self.T_mult + self.T_up
        else:
            if epoch >= self.T_0:
                if self.T_mult == 1:
                    self.T_cur = epoch % self.T_0
                    self.cycle = epoch // self.T_0
                else:
                    n = int(math.log((epoch / self.T_0 * (self.T_mult - 1) + 1), self.T_mult))
                    self.cycle = n
                    self.T_cur = epoch - self.T_0 * (self.T_mult ** n - 1) / (self.T_mult - 1)
                    self.T_i = self.T_0 * self.T_mult ** (n)
            else:
                self.T_i = self.T_0
                self.T_cur = epoch
                
        self.eta_max = self.base_eta_max * (self.gamma**self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

In [4]:
from torchvision import datasets, models, transforms
from torchvision.transforms import Resize, ToTensor, Normalize,CenterCrop

#원래는 transform으로 아주 다양한 방법들을 모두 적용해 보았지만 결국에는 대부분을 뺐을때 성능이 가장 잘 나왔다.
transform = transforms.Compose([
            ToTensor(),
    Normalize(mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246))
        ])
dataset = MaskBaseDataset(
    img_dir=cfg.img_dir,transform=transform
)


n_val = int(len(dataset) * 0.2)
n_train = len(dataset) - n_val
train_dataset, val_dataset = data.random_split(dataset, [n_train, n_val])


train_loader = data.DataLoader(
    train_dataset,
    batch_size=16,
    num_workers=4,
    shuffle=True
)

val_loader = data.DataLoader(
    val_dataset,
    batch_size=16,
    num_workers=4,
    shuffle=False
)

save_dir = os.path.join('model', 'efficient-b4')
model = EfficientNet.from_pretrained('efficientnet-b4')
model._fc = nn.Linear(1792, 18)
initialize_weights(model._fc)
model_ft = model.to(device)
criterion1 = nn.CrossEntropyLoss()
criterion2 = FocalLoss()
optimizer = AdamP(model.parameters(), lr=0.000001, betas=(0.9, 0.999), weight_decay=1e-4)
#scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5000, T_mult=2, eta_min=0.00005)
#scheduler = CosineAnnealingWarmUpRestart(optimizer, T_0=3000, T_mult=1, eta_max=0.001,  T_up=1000, gamma=0.5)


for epoch in range(5):
    model_ft.train()
    avg_loss = 0
    for inputs, mask_label,gender_label,age_label in tqdm(train_loader):
        inputs = inputs.to(device)
        mask_label = mask_label.to(device)
        gender_label = gender_label.to(device)
        age_label = age_label.to(device)
        outputs = model_ft(inputs)
        optimizer.zero_grad()
        
        loss1 = criterion1(outputs[:,:3],mask_label)
        #loss2 = criterion2(outputs[:,:3],mask_label)
        loss3 = criterion1(outputs[:,3:5],gender_label)
        #loss4 = criterion2(outputs[:,3:5],gender_label)
        loss5 = criterion1(outputs[:,5:8],age_label)
        #loss6 = criterion2(outputs[:,5:8],age_label)
        #loss = ((0.5*loss1)+(0.5*loss2)+(0.5*loss3)+(0.5*loss4)+(0.5*loss5)+(0.5*loss6))/3
        loss = (loss1+loss3+loss5)/3
        loss.backward()
        optimizer.step()
        scheduler.step()
        avg_loss+=loss.item()
    print(avg_loss/len(train_loader))


'''
        # val loop
        with torch.no_grad():
            print("Calculating validation results...")
            model.eval()
            val_loss_items = []
            val_acc_items = []
            figure = None
            for val_batch in val_loader:
                inputs, labels = val_batch
                inputs = inputs.to(device)
                labels = labels.to(device)
                outs = model(inputs)
                preds = torch.argmax(outs, dim=-1)
                loss_item = criterion(outs, labels).item()
                acc_item = (labels == preds).sum().item()
                val_loss_items.append(loss_item)
                val_acc_items.append(acc_item)
            val_loss = np.sum(val_loss_items) / len(val_loader)
            val_acc = np.sum(val_acc_items) / len(val_dataset)
            best_val_loss = min(best_val_loss, val_loss)
            if val_acc > best_val_acc:
                print(f"New best model for val accuracy : {val_acc:4.2%}! saving the best model..")
                torch.save(model.state_dict(), f"{save_dir}/best.pth")
                best_val_acc = val_acc
                counter = 0
            else:
                counter+=1
            torch.save(model.state_dict(), f"{save_dir}/last.pth")
            print(
                f"[Val] acc : {val_acc:4.2%}, loss: {val_loss:4.2} || "
                f"best acc : {best_val_acc:4.2%}, best loss: {best_val_loss:4.2}"
            )
            if counter > 2:
                print("Early Stopping...")
                break
'''


PyTorch version:[1.6.0].
device:[cuda:0].
Loaded pretrained weights for efficientnet-b4


  0%|          | 10/2835 [00:18<1:21:23,  1.73s/it]

Epoch[0/10](10/2835) || training loss 0.97 || training accuracy 3.33% || lr 1.0990000000000002e-05


  1%|          | 20/2835 [00:35<1:21:46,  1.74s/it]

Epoch[0/10](20/2835) || training loss 0.9685 || training accuracy 5.83% || lr 2.0980000000000002e-05


...

[Val] acc : 94.97%, loss: 0.62 || best acc : 94.97%, best loss: 0.62


  0%|          | 10/2835 [00:17<1:22:06,  1.74s/it]

Epoch[5/10](10/2835) || training loss 0.6017 || training accuracy 92.50% || lr 2.2939370003620206e-05


  0%|          | 14/2835 [00:26<1:28:02,  1.87s/it]


KeyboardInterrupt: 

In [5]:
test_dir = '../input/data/eval'


from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize,CenterCrop
class TestDataset(data.Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)


# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')


# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
'''
transform = Compose([
    Resize((512, 384), Image.BILINEAR),
    Normalize(mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246),max_pixel_value=255.0,p=1.0),
    ToTensorV2(p=1.0)
])
'''
transform = transforms.Compose([
    ToTensor(),
    Normalize(mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246)),
])
dataset = TestDataset(image_paths, transformer)

loader = data.DataLoader(
    dataset,
    shuffle=False
)
# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)


model_ft.eval()


# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in tqdm(loader):
    with torch.no_grad():
        images = images.to(device)
        pred = model_ft(images)
        mask_pred = pred[:,:3].argmax(dim=-1)
        gender_pred = pred[:,3:5].argmax(dim=-1)
        age_pred = pred[:,5:].argmax(dim=-1)
        pred = (mask_pred * 6 + gender_pred * 3 + age_pred).cpu().numpy()
        all_predictions.extend(pred)
submission['ans'] = all_predictions



# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)

100%|██████████| 12600/12600 [14:53<00:00, 14.10it/s]
